# Week 13. In-Class Practice
## 修真聊天群的 word2vec 實作

In [1]:
import jieba
import numpy as np
import gensim
import warnings
from ipywidgets import IntProgress
from hanziconv import HanziConv
from IPython.display import display

C:\Users\kevink556\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
file_train_read = []
with open('./dataset/xiuzhenliaotianqun.txt', encoding='utf-8') as file_train_raw:
    for line in file_train_raw:
        if line.strip() != '':
            file_train_read.append(HanziConv.toTraditional(line.strip()))

print("Text lines:", len(file_train_read))

Text lines: 213871


In [3]:
stopwords = set()
with open('./dataset/stopwords.txt', encoding='utf-8') as stopword_file:
    for words in stopword_file:
        stopwords.add(HanziConv.toTraditional(words.strip()))

print("Stopwords number:", len(stopwords))

Stopwords number: 745


In [4]:
progress = IntProgress(min=0, max=len(file_train_read))
progress.value = 0
progress.description = "[ %s / %s ]"%(str(progress.value), str(progress.max))
display(progress)

file_train_seg = []
for i in range(len(file_train_read)):
    file_train_seg.append([' '.join([word for word in jieba.cut(file_train_read[i], cut_all=False)
                                            if word not in stopwords])])
    progress.value +=1
    progress.description = "[ %s / %s ]"%(str(progress.value), str(progress.max))

IntProgress(value=0, description='[ 0 / 213871 ]', max=213871)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\KEVINK~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.077 seconds.
Prefix dict has been built succesfully.


In [5]:
# 將jieba的斷詞產出存檔
file_seg_word_done_path = 'corpus_seg_done.txt'
with open(file_seg_word_done_path, 'wb') as f:
    for i in range(len(file_train_seg)):
        f.write((file_train_seg[i][0] + '\n').encode('utf-8'))

# 檢視斷詞 jieba 的結果
def print_list_chinese(list):
    for i in range(len(list)):
        print(list[i])
        
print_list_chinese(file_train_seg[3])

季節 江南 地區 晝夜 溫差 變 很大 白天 還穿 褲衩 熱成 狗 晚上 縮 窩 裏 凍成 寒號 鳥


In [6]:
from gensim.models import word2vec
sentences = word2vec.LineSentence(file_seg_word_done_path)
model = word2vec.Word2Vec(sentences, size=250)
model.save("word2vec.model")

C:\Users\kevink556\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


In [7]:
# 可以用 model.wv.syn0 檢視詞向量，因為一個詞有250個維度，全部列出過於冗長.....
# 這邊僅僅呈現 10 個詞的 10 個維度

model.wv.syn0[0:10,0:10]

C:\Users\kevink556\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  after removing the cwd from sys.path.


array([[-0.7968492 , -0.16908197, -0.80101156, -0.78538275,  0.47682828,
        -0.01210073,  0.4343004 , -0.8885662 , -0.7434057 , -0.69766587],
       [ 0.5947968 , -0.6265896 ,  0.4628336 ,  0.05538532, -0.38051885,
        -0.68068534, -1.134348  ,  0.33785427,  0.14968921, -0.50054204],
       [ 0.28178832,  0.64006305, -0.21529683, -1.3555954 ,  1.0761099 ,
        -0.20608024, -0.6259862 ,  0.30736363, -1.2582827 , -0.52524406],
       [-0.467692  , -0.01088117,  0.94149107, -0.7858169 , -0.4444069 ,
         0.02835883, -0.25186068,  0.829317  , -0.08071167, -0.74918634],
       [ 0.13304971,  0.26200008, -1.0281827 ,  0.57304007, -0.23213628,
         0.37781435,  1.4464221 ,  1.6496657 , -1.3580114 ,  1.9172757 ],
       [ 1.3082304 ,  1.5606303 , -0.73164725, -0.6006969 ,  0.6776822 ,
         1.2346125 ,  0.42586622, -0.61109245, -0.350493  , -0.752032  ],
       [ 0.89260757,  0.64374447,  2.2064142 ,  1.6680992 , -0.6439324 ,
        -1.128948  , -1.5579499 , -1.249654  

In [8]:
# 可檢視第 996~1000 個字詞是什麼

for i in range(995,1000):
    print(model.wv.index2word[i])

高升
丹藥
起身
路
盔甲


In [9]:
# 顯示空間距離相近的詞
print("宋書航 相近詞：", [i[0] for i in model.similar_by_vector('宋書航')], '\n')
print("修真 相近詞：", [i[0] for i in model.similar_by_vector('修真')], '\n')
print("法術 相近詞：", [i[0] for i in model.similar_by_vector('法術')], '\n')

# 顯示相近詞和詞向量，直接使用 similar_by_vector() 就可以了
model.similar_by_vector('宋書航')

C:\Users\kevink556\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).
  


宋書航 相近詞： ['書航', '赤瞳', '宋航', '將書航', '瞭書航', '葉思', '李教員', '小彩', '葉師姐', '蔥娘'] 

修真 相近詞： ['資料', '加入', '是否', '本次', '設定', '權限', '冒險', '私人', '共享', '更改'] 

法術 相近詞： ['道術', '能力', '小法術', '催眠', '術', '強力', '天賦', '幻術', '秘術', '秘法'] 



C:\Users\kevink556\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
C:\Users\kevink556\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\kevink556\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).
  after removing the cwd from sys.path.
C:\Users\kevink556\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: Deprecat

[('書航', 0.6530104279518127),
 ('赤瞳', 0.6222689151763916),
 ('宋航', 0.5915800333023071),
 ('將書航', 0.5310773253440857),
 ('瞭書航', 0.5094061493873596),
 ('葉思', 0.5038828253746033),
 ('李教員', 0.46544134616851807),
 ('小彩', 0.4620957374572754),
 ('葉師姐', 0.45875030755996704),
 ('蔥娘', 0.45868104696273804)]

回過頭來看一下詞向量模型的結果。

首先我們用咱們男主角的名字「宋書航」丟進去測試，原則上要跑出一些主線人物的名字，畢竟他們伴隨著男主角成長，比較可能存在類似的脈絡中而被模型捕捉到。乍看也還算合理，不過「瞭書航」、「由書航」這二個詞很明顯地就是由簡轉繁引發的斷詞失誤了。

第二個嘗試看看「修真」的相近詞，跑出來「資料」、「權限」、「共享」等詞彙，再度懷疑是斷詞引擎或者詞向量模型出了問題。其實不然，有看過這部小說的同學就會知道這部作品是多麼的不(ㄋㄠˇ) 落(ㄉㄨㄥˋ) 俗(ㄉㄚˋ) 套(ㄎㄞ)。這個例子再度告訴我們，Domain Knowlwdge 的重要性啊 XDDD。

最後看看「法術」的相近詞，嗯，終於合乎我們的預期。這些字詞都是玄幻修真類小說的常用語彙，成功～

---------

不過還有一個小麻煩，同樣關乎斷詞引擎的失誤，就是當我們輸入一個很重要的角色「白前輩」進去搜尋相近詞時，居然找不到！

In [107]:
model.similar_by_vector('白前輩')

KeyError: "word '白前輩' not in vocabulary"

這邊我們有兩個問題要修正，一是關於簡轉繁後的判讀失誤，二是人名的錯誤斷詞。第一個問題，我們可以先讓斷詞引擎斷完，我們再翻譯語料庫中的詞彙。第二個問題我們則要加入相關字詞的字庫來調整斷詞。